<a href="https://colab.research.google.com/github/bryancev/Data_Engineer_traineeship/blob/main/Retail_store_sales_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests

# Получение "прямой" ссылки
public_url = "https://disk.yandex.ru/d/5yhVs9f0xe4w-Q"
api_response = requests.get(f"https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={public_url}")
api_response.raise_for_status()

file_response = requests.get(api_response.json()['href'])
file_response.raise_for_status()

# Сохранение в файл
with open("retail_store_sales.csv", "wb") as f:
    f.write(file_response.content)

# 1. Загрузка и предварительная обработка данных

## 1.1. Загрузка и вывод схемы:
Загрузите файл retail_store_sales.csv.  Выведите первые 5 строк загруженного DataFrame и его схему (df.printSchema()).

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Создаем SparkSession
spark = SparkSession.builder \
         .appName("retail_store_sales_analysis") \
         .getOrCreate()

df = spark.read \
    .option("header", True) \
    .option("sep", ",") \
    .csv("retail_store_sales.csv")

df.show(5, truncate=False)

In [ ]:
print(f"Количество строк в исходном DataFrame: {df.count()}")
print(f"""Количество уникальных Transaction ID: {df.select("Transaction ID").distinct().count()}""")

Количество строк в исходном DataFrame: 12575
Количество уникальных Transaction ID: 12575


## 1.2. Очистка названий столбцов:
Преобразуйте названия всех столбцов к единому регистру - **snake_case**.  Выведите обновленную схему DataFrame  или названия столбцов, чтобы убедиться в изменении названий.

In [ ]:
new_column_names = [col.replace(" ", "_").lower() for col in df.columns]

# Переименовывание столблов
df = df.toDF(*new_column_names)

print("Первые 5 записей:")
df.show(5, truncate=False)

Первые 5 записей:
+--------------+-----------+-------------+------------+--------------+--------+-----------+--------------+--------+----------------+----------------+
|transaction_id|customer_id|category     |item        |price_per_unit|quantity|total_spent|payment_method|location|transaction_date|discount_applied|
+--------------+-----------+-------------+------------+--------------+--------+-----------+--------------+--------+----------------+----------------+
|TXN_6867343   |CUST_09    |Patisserie   |Item_10_PAT |18.5          |10.0    |185.0      |Digital Wallet|Online  |2024-04-08      |True            |
|TXN_3731986   |CUST_22    |Milk Products|Item_17_MILK|29.0          |9.0     |261.0      |Digital Wallet|Online  |2023-07-23      |True            |
|TXN_9303719   |CUST_02    |Butchers     |Item_12_BUT |21.5          |2.0     |43.0       |Credit Card   |Online  |2022-10-05      |False           |
|TXN_9458126   |CUST_06    |Beverages    |Item_16_BEV |27.5          |9.0     |247

## 1.3. Преобразование типов данных:
Проанализируйте к каким типам данных относятся данные в столбцах и приведите столбец к соответствующему типу. Убедитесь, что некорректные или отсутствующие значения преобразуются в null в соответствующих типах данных.

In [ ]:
print("Схема данных до преобразования:")
df.printSchema()

Схема данных до преобразования:
root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- item: string (nullable = true)
 |-- price_per_unit: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- total_spent: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- location: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- discount_applied: string (nullable = true)



In [ ]:
# Преобразования с улучшенной обработкой NULL

def safe_cast(column, target_type):
    """Безопасное приведение типов с обработкой NULL и сохранением имени столбца"""
    return F.when(
        F.lower(F.trim(F.col(column))).isin("", "null", "na", "none", "nan"),
        F.lit(None)
    ).otherwise(
        F.col(column).cast(target_type)
    ).alias(column)

In [ ]:
# Применение преобразования:

df = df.select([
        # Строковые колонки
        safe_cast("transaction_id", "string"),
        safe_cast("customer_id", "string"),
        safe_cast("category", "string"),
        safe_cast("item", "string"),
        safe_cast("payment_method", "string"),
        safe_cast("location", "string"),

        # Числовые колонки
        safe_cast("price_per_unit", "double"),
        safe_cast("quantity", "integer"),
        safe_cast("total_spent", "double"),

        # Обработка даты
        F.to_date(
            F.when(
                F.lower(F.trim(F.col("transaction_date"))).isin("", "null", "na", "none"),
                F.lit(None)
            ).otherwise(F.col("transaction_date")),
            "yyyy-MM-dd"
        ).alias("transaction_date"),

        # Обработка булевых значений (исправлено на discount_applied)
        F.when(
            F.lower(F.trim(F.col("discount_applied"))).isin("true", "yes", "1", "t", "y"),
            True
        ).when(
            F.lower(F.trim(F.col("discount_applied"))).isin("false", "no", "0", "f", "n"),
            False
        ).otherwise(None).alias("discount_applied")
])

In [ ]:
print("Количество null значений:")
df.select([F.sum(F.isnull(c).cast("int")).alias(c) for c in df.columns]).show()

Количество null значений:
+--------------+-----------+--------+----+--------------+--------+--------------+--------+-----------+----------------+----------------+
|transaction_id|customer_id|category|item|payment_method|location|price_per_unit|quantity|total_spent|transaction_date|discount_applied|
+--------------+-----------+--------+----+--------------+--------+--------------+--------+-----------+----------------+----------------+
|             0|          0|       0|1213|             0|       0|           609|     604|        604|               0|            4199|
+--------------+-----------+--------+----+--------------+--------+--------------+--------+-----------+----------------+----------------+



In [ ]:
# Задание безопасной схемы - с учетом наличия/отсутствия null значений после преобразования

safe_schema = StructType([
    StructField("transaction_id", StringType(), False),
    StructField("customer_id", StringType(), False),
    StructField("category", StringType(), False),
    StructField("item", StringType(), True),
    StructField("payment_method", StringType(), False),
    StructField("location", StringType(), False),
    StructField("price_per_unit", DoubleType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("total_spent", DoubleType(), True),
    StructField("transaction_date", DateType(), False),
    StructField("discount_applied", BooleanType(), True)
])

# Заменяем схему у существующего DataFrame
df = spark.createDataFrame(df.rdd, safe_schema)

In [ ]:
print("Схема данных после преобразования:")
df.printSchema()

Схема данных после преобразования:
root
 |-- transaction_id: string (nullable = false)
 |-- customer_id: string (nullable = false)
 |-- category: string (nullable = false)
 |-- item: string (nullable = true)
 |-- payment_method: string (nullable = false)
 |-- location: string (nullable = false)
 |-- price_per_unit: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_spent: double (nullable = true)
 |-- transaction_date: date (nullable = false)
 |-- discount_applied: boolean (nullable = true)



In [ ]:
print("Пример данных:")
df.show(20, truncate=False)

Пример данных:
+--------------+-----------+-----------------------------+------------+--------------+--------+--------------+--------+-----------+----------------+----------------+
|transaction_id|customer_id|category                     |item        |payment_method|location|price_per_unit|quantity|total_spent|transaction_date|discount_applied|
+--------------+-----------+-----------------------------+------------+--------------+--------+--------------+--------+-----------+----------------+----------------+
|TXN_6867343   |CUST_09    |Patisserie                   |Item_10_PAT |Digital Wallet|Online  |18.5          |10      |185.0      |2024-04-08      |true            |
|TXN_3731986   |CUST_22    |Milk Products                |Item_17_MILK|Digital Wallet|Online  |29.0          |9       |261.0      |2023-07-23      |true            |
|TXN_9303719   |CUST_02    |Butchers                     |Item_12_BUT |Credit Card   |Online  |21.5          |2       |43.0       |2022-10-05      |false  

# 2. Очистка и валидация данных

2.1. Восстановление отсутствующих **item**:

*   Так как данные статические для каждого товара, то  составьте справочник товаров в отдельный DataFrame с **Category**, **Item** и **Rrice Rer Unit**.
*   Для транзакций, где отсутствует название товара , но имеется категория и цена , попытайтесь определить название товара, путём объединения (join) с загруженным справочником товаров. Выведите 20 строк, демонстрирующих восстановленные значения.


In [ ]:
# 1. Справочник товаров из имеющихся данных
catalog = (
    df.filter(
        F.col("item").isNotNull() &
        F.col("category").isNotNull() &
        F.col("price_per_unit").isNotNull()
    )
    .select(
        F.col("category").alias("category_catalog"),
        F.col("item").alias("item_catalog"),
        F.col("price_per_unit").alias("price_per_unit_catalog")
    )
    .distinct()
)

print("Справочник товаров (первые 5 записей):")
catalog.show(5, truncate=False)

Справочник товаров (первые 5 записей):
+----------------------------------+------------+----------------------+
|category_catalog                  |item_catalog|price_per_unit_catalog|
+----------------------------------+------------+----------------------+
|Beverages                         |Item_16_BEV |27.5                  |
|Beverages                         |Item_7_BEV  |14.0                  |
|Electric household essentials     |Item_23_EHE |38.0                  |
|Milk Products                     |Item_16_MILK|27.5                  |
|Computers and electric accessories|Item_1_CEA  |5.0                   |
+----------------------------------+------------+----------------------+
only showing top 5 rows



In [ ]:
# Поиск товаров с несколькими ценами

(catalog
    .groupBy("category_catalog", "item_catalog")
    .agg(F.count("price_per_unit_catalog").alias("price_count"))
    .filter(F.col("price_count") > 1)
    .orderBy(F.col("price_count").desc())
    .show(5, truncate=False)
)

+----------------+------------+-----------+
|category_catalog|item_catalog|price_count|
+----------------+------------+-----------+
+----------------+------------+-----------+



In [ ]:
# 2. Заполнение пропусков в item

df_filled = (
    df.join(
        catalog,
        (F.col("category") == F.col("category_catalog")) &
        (F.col("price_per_unit") == F.col("price_per_unit_catalog")),
        "left"
    )
    .withColumn(
        "item",
        F.coalesce(
            F.col("item"),
            F.col("item_catalog")  # если item пуст – берём из справочника
        )
    )
    .drop("category_catalog", "item_catalog", "price_per_unit_catalog")  # удаление столбцов справочника
)

In [ ]:
# 3. Анализ результатов

print("Анализ восстановления данных:")
print(f"Исходное количество пропусков в item: {df.filter(F.col('item').isNull()).count()}")
print(f"Количество пропусков в item после заполнения: {df_filled.filter(F.col('item').isNull()).count()}")

Анализ восстановления данных:
Исходное количество пропусков в item: 1213
Количество пропусков в item после заполнения: 609


In [ ]:
print("После восстановления данных (первые 20 записей):")
df_filled.show(20, truncate=False)

После восстановления данных (первые 20 записей):
+--------------+-----------+-----------------------------+------------+--------------+--------+--------------+--------+-----------+----------------+----------------+
|transaction_id|customer_id|category                     |item        |payment_method|location|price_per_unit|quantity|total_spent|transaction_date|discount_applied|
+--------------+-----------+-----------------------------+------------+--------------+--------+--------------+--------+-----------+----------------+----------------+
|TXN_4575373   |CUST_05    |Food                         |Item_6_FOOD |Digital Wallet|Online  |12.5          |7       |87.5       |2022-10-02      |false           |
|TXN_4396807   |CUST_17    |Electric household essentials|Item_13_EHE |Digital Wallet|In-store|23.0          |1       |23.0       |2022-02-07      |false           |
|TXN_4206593   |CUST_01    |Furniture                    |Item_21_FUR |Digital Wallet|Online  |35.0          |NULL    |NU

## 2.2. Восстановление Total Spent:
Найдите все транзакции, с пропусками в общей сумме и обновите ее, пересчитав её как **quantity * price_per_unit**  для всех записей.

In [ ]:
print(f"Исходное количество пропусков в total_spent: {df_filled.filter(F.col('total_spent').isNull()).count()}")

Исходное количество пропусков в total_spent: 604


In [ ]:
# Заполнение пропусков в total_spent

df_filled = (
    df_filled.join(
        catalog,
        (F.col("category") == F.col("category_catalog")) &
        (F.col("item") == F.col("item_catalog")),
        "left"
    )
    .withColumn(
        "total_spent",
        F.coalesce(
            F.col("total_spent"),
            F.round(F.col("quantity") * F.col("price_per_unit_catalog"), 1)   # Иначе вычисляем по цене из справочник
        )
    )
    .drop("category_catalog", "item_catalog", "price_per_unit_catalog")       # удаление столбцов справочника
)

In [ ]:
print(f"Количество пропусков в total_spent после заполнения: {df_filled.filter(F.col('total_spent').isNull()).count()}")

Количество пропусков в total_spent после заполнения: 604


**Восстановить total_spent не удалось**

## 2.3. Заполнение отсутствующих Quantity:

*   Для транзакций, где отсутствуют значения о количестве проданного товара , но имеются сумма транзакции и цена за товар , вычислите количество проданного товара и заполните пропущенные значения. Результат приведите к целому числу.



In [ ]:
print(f"Исходное количество пропусков в quantity: {df_filled.filter(F.col('quantity').isNull()).count()}")

Исходное количество пропусков в quantity: 604


In [ ]:
# Сначала создаем DataFrame с вычисленными значениями quantity

df_calc_quantity = (
    df_filled.filter(
        F.col("quantity").isNull() &
        F.col("total_spent").isNotNull() &
        F.col("price_per_unit").isNotNull() &
        (F.col("price_per_unit") != 0)
    )
    .withColumn(
        "calc_quantity",
        F.round(F.col("total_spent") / F.col("price_per_unit"), 0).cast("integer")
    )
)

# Объединяем с исходным DataFrame и заполняем пропуски
df_filled = (
    df_filled
    .join(
        df_calc_quantity.select("transaction_id", "calc_quantity"),
        on="transaction_id",
        how="left"
    )
    .withColumn(
        "quantity",
        F.coalesce(F.col("quantity"), F.col("calc_quantity"))
    )
    .drop("calc_quantity")  # удаление временного столбеца
)

In [ ]:
print(f"Количество пропусков в quantity после заполнения: {df_filled.filter(F.col('quantity').isNull()).count()}")

Количество пропусков в quantity после заполнения: 604


**Восстановить quantity не удалось**

## 2.3. Заполнение отсутствующих Price Rer Unit:

*   Аналогично, если  отсутствует цена за единицу товара , но общая сумма и количество имеются, вычислите цену за единицу и заполните пропущенные значения. Округлите до двух знаков после запятой. Выведите 20 строк, демонстрирующих заполненные значения.

In [ ]:
print(f"Исходное количество пропусков в price_per_unit: {df_filled.filter(F.col('price_per_unit').isNull()).count()}")

Исходное количество пропусков в price_per_unit: 609


In [ ]:
# Сначала создаем DataFrame с вычисленными значениями quantity

df_calc_price = (
    df_filled.filter(
        F.col("price_per_unit").isNull() &
        F.col("total_spent").isNotNull() &
        F.col("quantity").isNotNull()    &
        (F.col("quantity") != 0)
    )
    .withColumn(
        "calc_price",
        F.round(F.col("total_spent") / F.col("quantity"), 2)
    )
)

# Объединяем с исходным DataFrame и заполняем пропуски
df_filled = (
    df_filled
    .join(
        df_calc_price.select("transaction_id", "calc_price"),
        on="transaction_id",
        how="left"
    )
    .withColumn(
        "price_per_unit",
        F.coalesce(F.col("price_per_unit"), F.col("calc_price"))
    )
    .drop("calc_price")  # удаление временного столбеца
)

In [ ]:
print(f"Количество пропусков в price_per_unit после заполнения: {df_filled.filter(F.col('price_per_unit').isNull()).count()}")

Количество пропусков в price_per_unit после заполнения: 0


## 2.4. Удалите оставшийся строки с пропусками

в Category, Quantity ,Total Spent и Price Rer Unit

In [ ]:
# Список столбцов - по заданию
columns_to_drop = ["category", "quantity", "total_spent", "price_per_unit"] # "item",

# Удаляем строки, где есть NULL в указанных столбцах
df_filled = df_filled.dropna(subset=columns_to_drop)

In [ ]:
print("Количество null значений после восстановления:")
df_filled.select([F.sum(F.isnull(c).cast("int")).alias(c) for c in df_filled.columns]).show()

Количество null значений после восстановления:
+--------------+-----------+--------+----+--------------+--------+--------------+--------+-----------+----------------+----------------+
|transaction_id|customer_id|category|item|payment_method|location|price_per_unit|quantity|total_spent|transaction_date|discount_applied|
+--------------+-----------+--------+----+--------------+--------+--------------+--------+-----------+----------------+----------------+
|             0|          0|       0| 609|             0|       0|             0|       0|          0|               0|            3988|
+--------------+-----------+--------+----+--------------+--------+--------------+--------+-----------+----------------+----------------+



In [ ]:
# Вывод результатов

initial_count = df.count()
filled_count = df_filled.count()
pct_change = ((initial_count - filled_count) / initial_count) * 100

print(f"Количество строк в исходном DataFrame: {initial_count}")
print(f"Количество строк после восстановления: {filled_count}")

print(f"Количество удаленных строк: {initial_count - filled_count}")
print(f"Процент удаленных строк: {pct_change:.2f}%")

Количество строк в исходном DataFrame: 12575
Количество строк после восстановления: 11971
Количество удаленных строк: 604
Процент удаленных строк: 4.80%


# 3. Разведочный анализ данных

In [ ]:
print("Первые 5 записей:")
df_filled.show(5, truncate=False)

Первые 5 записей:
+--------------+-----------+----------------------------------+-----------+--------------+--------+--------------+--------+-----------+----------------+----------------+
|transaction_id|customer_id|category                          |item       |payment_method|location|price_per_unit|quantity|total_spent|transaction_date|discount_applied|
+--------------+-----------+----------------------------------+-----------+--------------+--------+--------------+--------+-----------+----------------+----------------+
|TXN_9049596   |CUST_11    |Furniture                         |Item_2_FUR |Credit Card   |Online  |6.5           |4       |26.0       |2023-07-26      |false           |
|TXN_3593095   |CUST_03    |Computers and electric accessories|Item_19_CEA|Digital Wallet|Online  |32.0          |3       |96.0       |2024-05-31      |NULL            |
|TXN_1018578   |CUST_10    |Computers and electric accessories|Item_10_CEA|Cash          |Online  |18.5          |3       |55.5     

## 3.1 Рассчитайте общее количество проданных единиц товара  для каждой категории. Определите **Топ-5 категорий** по общему количеству проданных единиц.

In [ ]:
# количество проданных единиц для каждой категории

(df_filled.groupBy(F.col("category"))
    .agg(F.sum("quantity").alias("total_quantity"))
    .orderBy(F.col("total_quantity").desc())
    .show(5, truncate=False)
)

+-----------------------------+--------------+
|category                     |total_quantity|
+-----------------------------+--------------+
|Furniture                    |8462          |
|Food                         |8387          |
|Beverages                    |8358          |
|Milk Products                |8339          |
|Electric household essentials|8309          |
+-----------------------------+--------------+
only showing top 5 rows



## 3.2. Анализ среднего чека:

*   Рассчитайте среднее значение **Total Spent** для каждого метода оплаты. Округлите до двух знаков после запятой.

*   Рассчитайте среднее значение **Total Spent** для каждой места где прошла оплата. Округлите до двух знаков после запятой.

In [ ]:
# Среднее значение total_spent для каждого метода оплаты

(
    df_filled.groupBy("payment_method")
    .agg(
        F.round(F.avg("total_spent"), 2).alias("avg_total_spent")
        )
    .orderBy(F.desc("avg_total_spent"))
    .show(truncate=False)
)

+--------------+---------------+
|payment_method|avg_total_spent|
+--------------+---------------+
|Cash          |131.05         |
|Credit Card   |129.13         |
|Digital Wallet|128.72         |
+--------------+---------------+



In [ ]:
# Среднее значение total_spent для каждого места где прошла оплата

(
    df_filled.groupBy("location")
    .agg(
        F.round(F.avg("total_spent"), 2).alias("avg_total_spent")
        )
    .orderBy(F.desc("avg_total_spent"))
    .show(truncate=False)
)

+--------+---------------+
|location|avg_total_spent|
+--------+---------------+
|Online  |130.42         |
|In-store|128.86         |
+--------+---------------+



# 4. Генерация признаков

## 4.1. Временные признаки
Добавьте два новых столбца на основе Transaction Date:

*   **day_of_week**: День недели
*   **transaction_month**: Месяц транзакции

In [ ]:
# Добавляем временные признаки

df_filled = df_filled.withColumns({
    "day_of_week": F.dayofweek("transaction_date"),
    "transaction_month": F.month("transaction_date")
})

# выборка только некоторых колонок
df_filled.select("transaction_date", "day_of_week", "transaction_month").show(5, truncate=False)

+----------------+-----------+-----------------+
|transaction_date|day_of_week|transaction_month|
+----------------+-----------+-----------------+
|2023-07-26      |4          |7                |
|2024-05-31      |6          |5                |
|2023-01-17      |3          |1                |
|2024-01-13      |7          |1                |
|2024-03-18      |2          |3                |
+----------------+-----------+-----------------+
only showing top 5 rows



## 4.2. Продажи по дням недели
Рассчитайте среднюю сумму продаж (Total Spent) для каждого дня недели. Выведите результаты, отсортированные по дням недели

In [ ]:
# Cредняя сумма продаж для каждого дня недели

(
    df_filled.groupBy("day_of_week")
    .agg(F.avg("total_spent").alias("avg_total_spent"))
    .orderBy("day_of_week")
    .show(truncate=False)
)

+-----------+------------------+
|day_of_week|avg_total_spent   |
+-----------+------------------+
|1          |130.17914746543778|
|2          |125.56894519740719|
|3          |129.51061320754718|
|4          |126.81589713617767|
|5          |129.275486152033  |
|6          |134.6362058993638 |
|7          |131.49032258064517|
+-----------+------------------+



## 4.3.Продажи по месяцам
Рассчитайте среднюю сумму продаж (Total Spent)  для каждого месяца. Выведите результаты, отсортированные по месяцам

In [ ]:
# Cредняя сумма продаж для каждого месяца

(
    df_filled.groupBy("transaction_month")
    .agg(F.avg("total_spent").alias("avg_total_spent"))
    .orderBy("transaction_month")
    .show(truncate=False)
)

+-----------------+------------------+
|transaction_month|avg_total_spent   |
+-----------------+------------------+
|1                |134.68803088803088|
|2                |130.66048034934497|
|3                |126.83108808290156|
|4                |131.8137460650577 |
|5                |127.39723926380368|
|6                |130.94954591321897|
|7                |126.57266602502406|
|8                |124.28175403225806|
|9                |131.4471544715447 |
|10               |127.8517130620985 |
|11               |128.78578947368422|
|12               |133.15041067761808|
+-----------------+------------------+



## 4.4. Признаки клиента

Рассчитайте customer_lifetime_value (CLV) для каждого клиента как общую сумму (Total Spent), потраченную этим клиентом за все транзакции. Выведите Топ-10 клиентов по их CLV (customer_id и их CLV).

In [ ]:
# CLV для каждого клиента

(
    df_filled.groupBy(F.col("customer_id"))
    .agg(F.sum("total_spent").alias("clv"))
    .orderBy(F.desc("clv"))
    .show(10, truncate=False)
)

+-----------+-------+
|customer_id|clv    |
+-----------+-------+
|CUST_24    |68452.0|
|CUST_08    |67351.5|
|CUST_05    |66974.5|
|CUST_16    |65570.5|
|CUST_13    |65037.0|
|CUST_23    |64507.0|
|CUST_10    |63155.5|
|CUST_15    |63117.5|
|CUST_21    |62933.0|
|CUST_02    |62046.5|
+-----------+-------+
only showing top 10 rows

